In [ ]:
from bottle import Bottle, route, run, template, get, post, debug, static_file, request, redirect, response
import time
import random
import string
import logging
import logging.handlers
import t_test

log = logging.getLogger('bottle')
log.setLevel('INFO')
h = logging.handlers.TimedRotatingFileHandler(
    'logs/nlog', when='midnight', backupCount=9999)
f = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
h.setFormatter(f)
log.addHandler(h)


secretKey = "vcxv&TTFS98v334sd123fsdhkldasdjghd6DFOUFYWE&FY"


from users import users
from statics import messDict

app = Bottle()


@route('/static/<path:path>', name='static')
def static(path):
    return static_file(path, root='./static')


def checkAuth():
    loginName = request.get_cookie("user", secret=secretKey)
    randStr = request.get_cookie("randStr", secret=secretKey)
    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))
    if (loginName in users) and \
        (users[loginName].get("randStr", "") == randStr) and \
        (users[loginName]["loggedIn"] == True) and \
        (time.time() - users[loginName]["lastSeen"] < 3600):
        users[loginName]["lastSeen"] = time.time()
        return loginName
    return redirect('/login')


@route('/login')
@route('/login/')
@route('/login', method='POST')
def login():
    loginName = request.forms.getunicode('inputEmail', default=False)
    password = request.forms.getunicode('inputPassword', default=False)
    randStr = ''.join(random.choice(
        string.ascii_uppercase + string.digits) for _ in range(18))
    
    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))
    
    if (loginName in users) and users[loginName]["password"] == password:
        response.set_cookie("user", loginName, secret=secretKey)
        response.set_cookie("randStr", randStr, secret=secretKey)
        users[loginName]["loggedIn"] = True
        users[loginName]["randStr"] = randStr
        users[loginName]["lastSeen"] = time.time()
        
        redirect('/index')
        return True
    else:
        return template('login')
    return template('login')


@route('/')
@route('/index')
@route('/index/')
@route('/index/<message>')
def index(message=''):
    loginName = checkAuth()
    return template('index', message=messDict.get(message, ""), loginName=loginName)

@route('/login/')
@route('/login')
def login():
    return template("login")

@route('/register')
@route('/register/')
def regiser():
    return template("register")

@route('/one_sample_ttest')
@route('/one_sample_ttest/')
def one_sample_ttest():
    sample_mean = request.forms.get("inputSampleMean")
    sample_std = request.forms.get("inputSampleStd")
    test_type = request.forms.get("tail")
    hipothetical_mean = request.forms.get("inputHypothesizedMean")
    N = request.forms.get("inputSampleSize")
    alpha = request.forms.get("inputAplha")

    #compute = request.forms.get()
    
    #if compute:


    #results = t_test.one_sample_t_test(sample_mean, sample_std, hipothetical_mean, N)


    return template("one_sample_ttest")

@route('/static/<path:path>', name='static')
def static(path):
    return static_file(path, root='./static')

# Debug = True - changes in template are visible immidietly
run(host='localhost', port=8012, debug=True)

Bottle v0.12.19 server starting up (using WSGIRefServer())...
Listening on http://localhost:8012/
Hit Ctrl-C to quit.

127.0.0.1 - - [16/Jan/2022 13:30:33] "GET / HTTP/1.1" 303 0
127.0.0.1 - - [16/Jan/2022 13:30:33] "GET /login HTTP/1.1" 200 3543
127.0.0.1 - - [16/Jan/2022 13:30:33] "GET /favicon.ico HTTP/1.1" 404 742
127.0.0.1 - - [16/Jan/2022 13:30:35] "POST /login HTTP/1.1" 303 0
127.0.0.1 - - [16/Jan/2022 13:30:36] "GET /index HTTP/1.1" 200 6180
127.0.0.1 - - [16/Jan/2022 13:30:44] "GET /one_sample_ttest HTTP/1.1" 200 8560
127.0.0.1 - - [16/Jan/2022 13:31:35] "GET /one_sample_ttest HTTP/1.1" 200 8422
127.0.0.1 - - [16/Jan/2022 13:31:49] "GET /one_sample_ttest HTTP/1.1" 200 8423
127.0.0.1 - - [16/Jan/2022 13:31:55] "POST /one_sample_ttest HTTP/1.1" 405 749
127.0.0.1 - - [16/Jan/2022 13:32:02] "GET / HTTP/1.1" 200 6180
127.0.0.1 - - [16/Jan/2022 13:32:06] "GET /one_sample_ttest HTTP/1.1" 200 8423


In [ ]:
import t_test